In [1]:
#DB에서 영화cd,국적, 장르, 제작사에 따른 매출액 예측 할수 있는 데이터 프래임 만들기

In [81]:
import pandas as pd
from datetime import datetime
from DbConn import *
import numpy as np
from sklearn.model_selection import train_test_split

In [82]:
db= DbConn()
#GENRENM,COMPANYCD1

In [83]:
# sql = '''
# SELECT m.MOVIECD, m.MOVIENM, m.NATIONS, m.GENRENM, m.COMPANYCD1, b.OPENDT, b.DAILYDATE2, b.AUDIACC
# FROM MOVIEDETAIL m JOIN (
#    SELECT MOVIECD, OPENDT, DAILYDATE2, AUDIACC FROM BOXOFFICE WHERE DAILYDATE2 = OPENDT +60
# ) b ON m.MOVIECD = b.MOVIECD 
# '''

In [84]:
# sql2 = '''
# SELECT m.MOVIECD, m.MOVIENM, m.NATIONS, m.GENRENM, m.COMPANYCD1, b.OPENDT, b.DAILYDATE2, b.AUDIACC
# FROM MOVIEDETAIL m JOIN (
#    SELECT * FROM BOXOFFICE 
#    WHERE AUDIACC IN (SELECT MAX(AUDIACC) FROM BOXOFFICE GROUP BY MOVIECD)
# ) b ON m.MOVIECD = b.MOVIECD  
# '''

In [85]:
sql3 ='''
select 
CASE WHEN audiacc > 10000000 THEN 'A' 
when audiacc > 5000000 then 'B'
when audiacc > 1000000 then 'C'
ELSE 'D' END audiacc_grade, origin.*
from (
    SELECT b.*, m.MOVIENM , m.SHOWTM , m.NATIONS , m.GENRENM , m.DIRECTORS , m.AUDITS , m.COMPANYCD1
    FROM MOVIEDETAIL m JOIN (
    SELECT MOVIECD , MAX(AUDIACC) as audiacc
    FROM BOXOFFICE b2 GROUP BY MOVIECD 
    ) b ON m.MOVIECD = b.MOVIECD
) origin
'''

In [86]:
#dataFrame 객체 가져오기
query_result_df=pd.read_sql(sql3,db.connection)

In [87]:
db.disconnect()

In [88]:
query_result_df.keys()

Index(['AUDIACC_GRADE', 'MOVIECD', 'AUDIACC', 'MOVIENM', 'SHOWTM', 'NATIONS',
       'GENRENM', 'DIRECTORS', 'AUDITS', 'COMPANYCD1'],
      dtype='object')

In [89]:
X = query_result_df.drop(['MOVIECD','AUDIACC','MOVIENM','AUDITS','DIRECTORS','SHOWTM','AUDIACC'],axis=1)

In [90]:
y=query_result_df['AUDIACC_GRADE']
# y.head()

In [91]:
conditions = [
    (X['NATIONS'] == '한국'), # S
    (X['NATIONS'] == '미국'), # A
    (X['NATIONS'] == '일본') # A
]
nation_code = [1,2,3]
X['nation_code'] = np.select(conditions, nation_code, default=4)
X = X.drop(['NATIONS'], axis=1)

In [92]:
conditions_genr = [
    (X['GENRENM'] == '애니메이션'),
    (X['GENRENM'] == '드라마'),
    (X['GENRENM'] == '액션'),
    (X['GENRENM'] == '코미디'),
    (X['GENRENM'] == '멜로/로맨스'),
    (X['GENRENM'] == '범죄'),
    (X['GENRENM'] == '공포(호러)'),
    (X['GENRENM'] == '스릴러'),
    (X['GENRENM'] == '다큐멘터리'),
    (X['GENRENM'] == '미스터리'),
    (X['GENRENM'] == '어드벤처'),
    (X['GENRENM'] == '판타지'),
    (X['GENRENM'] == '사극'),
    (X['GENRENM'] == 'SF'),
    (X['GENRENM'] == '가족'),
    (X['GENRENM'] == '전쟁'),
    (X['GENRENM'] == '뮤지컬')
]
genre_code = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

X['genre_code'] = np.select(conditions_genr, genre_code, default=18)
X = X.drop(['GENRENM'], axis=1)

In [93]:

# df = pd.DataFrame(X['GENRENM'])
# df.apply(pd.value_counts)
# # # df
# # df.groupby('GENRENM').count()


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=999)

In [95]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

In [96]:
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train,y_train)

ValueError: could not convert string to float: 'D'

In [97]:
model.score(X_train,y_train)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [98]:
model.score(X_test,y_test)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [99]:
y_train.head()

1252    D
1097    D
477     D
1136    D
1361    D
Name: AUDIACC_GRADE, dtype: object